## 7.1 Amusement park data

### Load data

In [0]:
import pandas as pd
sat_df = pd.read_csv('http://bit.ly/PMR-ch7')
sat_df.head()

### Simulating the Amusement Park Data

In [0]:
import numpy as np
import pandas as pd
np.random.seed(8266)
n_resp = 500 # Number of survey responses

In [0]:
halo = np.random.normal(loc=0, scale=5, size=n_resp)

In [0]:
def generate_satisfaction_scores(mean, std, halo,
                                 score_range=(0, 100)):
  """Simulate satisfaction scores of a survey questions from a normal
  distributions.
  
  Args:
    mean: Numeric. The desired mean of the score distribution
    std: Numeric. The desired standard deviation of the score
      distribution
    translation: A constant added to each score
    halo: Array of numerics. The "halo" for each survey respondent,
      which accounts for individual respondents' likelihood to
      consistently respond higher or lower than average
    score_range: Tuple of numerics. The maximum and minimum
      scores, inclusive

  Returns:
    Array of int. Simulated satisfaction scores
  """
  # Draw scores from a normal distribution
  scores = np.random.normal(loc=mean, scale=std, size=len(halo))
  # Add the halo
  scores = scores + halo
  # Floor the scores so that they are all integers and clip to limit range
  scores = np.floor(scores)
  scores = np.clip(scores, score_range[0], score_range[1])

  return scores

In [0]:
rides = generate_satisfaction_scores(mean=81, std=3, halo=halo)
games = generate_satisfaction_scores(mean=75, std=7, halo=halo)
wait = generate_satisfaction_scores(mean=74, std=10, halo=halo)
clean = generate_satisfaction_scores(mean=86, std=2, halo=halo)

In [0]:
np.corrcoef(rides, games)

In [0]:
np.random.seed(82667)
distance = np.random.lognormal(mean=3, sigma=1, size=n_resp)
num_child = np.random.choice(a=range(6),
                             size=n_resp,
                             replace=True,
                             p=[0.3, 0.15, 0.25, 0.15, 0.1, 0.05])
weekend = np.random.choice(a=[True, False],
                           size=n_resp,
                           replace=True,
                           p=[0.5, 0.5])

In [0]:
overall = np.floor(0.7*(halo + 0.5*rides + 0.15*games + 0.3*wait
                   + 0.2*clean + 0.07*distance + 5*(num_child == 0)
                   + 0.3*wait*(num_child > 0)
                   + np.random.normal(loc=0, scale=7, size=n_resp)))
overall = np.clip(overall, 0, 100)

In [0]:
sat_df = pd.DataFrame({'is_weekend': weekend,
                       'num_child': num_child,
                       'distance': distance,
                       'rides': rides,
                       'games': games,
                       'wait': wait,
                       'clean': clean,
                       'overall': overall})
sat_df.is_weekend = sat_df.is_weekend.astype(pd.api.types.
                                             CategoricalDtype())

In [0]:
sat_df.head()

In [0]:
sat_df.to_csv(index=False)

## Fitting linear models with lm()

In [0]:
sat_df.describe().round(2)

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt


In [0]:
import seaborn as sns
sns.set_context('paper')
import matplotlib.pyplot as plt
g = sns.PairGrid(sat_df.replace({False: 0, True: 1}),
                 height=1.2, aspect=1.0)
g.map_upper(sns.scatterplot, linewidths=1, edgecolor="w", s=10,
            alpha=0.5)
g.map_diag(plt.hist)
g.map_lower(sns.kdeplot)

In [0]:
sat_df['log_dist'] = sat_df.distance.apply(np.log)

In [0]:
sat_df.log_dist.hist()
plt.xlabel('log distance')
plt.ylabel('Count')

In [0]:
sat_df.corr()

In [0]:
sat_df_corr = sat_df.corr()
sns.heatmap(sat_df_corr, annot=True, fmt=".2f",
            mask=~np.tri(sat_df_corr.shape[0], k=-1, dtype=bool),
            cbar=False)

In [0]:
sat_df.plot(kind='scatter', x='rides', y='overall')
plt.xlabel('Satisfaction with rides')
plt.ylabel('Satisfaction overall')

### Linear model with a single predictor

In [0]:
import statsmodels.formula.api as smf
smf.ols('overall ~ rides', data=sat_df).fit().summary()

In [0]:
-27.9869 + 1.2887*95

### ols objects

In [0]:
m1 = smf.ols('overall ~ rides', data=sat_df).fit()

In [0]:
# This plot and the one below it are identical
sat_df.plot(kind='scatter', x='rides', y='overall')
plt.plot(sat_df.rides, m1.predict(sat_df.rides))

In [0]:
from statsmodels.graphics import regressionplots
sat_df.plot(kind='scatter', x='rides', y='overall')
ax = plt.gca()
_ = regressionplots.abline_plot(model_results=m1, ax=ax)

In [0]:
m1.params

In [0]:
m1.predict({'rides': [95]})

In [0]:
m1.summary()

In [0]:
m1.conf_int()

In [0]:
np.corrcoef(sat_df.rides, sat_df.overall)**2

In [0]:
m1.resid.max(), m1.resid.min()

In [0]:
np.percentile(m1.resid, q=range(0,101,25))

In [0]:
plt.hist(m1.resid)
plt.xlabel('m1 residual value')
plt.ylabel('Count')

In [0]:
np.std(m1.resid)

## 7.2.5 Checking Model Fit

In [0]:
np.random.seed(8266)
x = np.random.normal(size=500)
y = x**2 + np.random.normal(size=500)
toy_model = smf.ols('y ~ x', data={'x': x, 'y': y}).fit()
toy_model.summary()

In [0]:
plt.scatter(x,y)
plt.plot(x, toy_model.predict({'x': x}))
plt.xlabel('x')
plt.ylabel('y')
plt.title('x vs y with fit line')

In [0]:
plt.scatter(x=toy_model.fittedvalues, y=toy_model.resid)
plt.xlabel('Fitted values')
plt.ylabel('Residuals')
plt.title('Predicted y values vs Residuals')

In [0]:
from statsmodels.graphics import gofplots
def plot_gof_figures(model):
  '''Plot a multipanel figure of goodness of fit plots'''
  sns.residplot(model.fittedvalues, model.resid, lowess=True)
  plt.xlabel('Fitted values')
  plt.ylabel('Residuals')
  plt.title('Residuals vs Fitted')
  plt.show()

  _ = gofplots.qqplot(model.resid, fit=True, line='45')
  plt.title('Normal Q-Q')
  plt.show()

  plt.scatter(model.fittedvalues, np.abs(model.resid)**.5)
  plt.xlabel('Fitted values')
  plt.ylabel('Square root of the standardized residuals')
  plt.title('Scale-Location')
  plt.show()

  regressionplots.plot_leverage_resid2(model)

In [0]:
plot_gof_figures(toy_model)

In [0]:
plot_gof_figures(m1)

In [0]:
sat_df.loc[[ 405, 48, 176]]

## 7.3 Fitting Linear Models with Multiple Predictors

In [0]:
m2 = smf.ols('overall ~ rides + games + wait + clean',
             data=sat_df).fit()
m2.summary()

In [0]:
np.std(m2.resid)

In [0]:
np.percentile(m2.resid, q=range(0,101,25))

In [0]:
!pip install python_marketing_research
from python_marketing_research_functions import chapter6

In [0]:
chapter6.plot_confidence_intervals(m2.params[1:],
                                   m2.conf_int().iloc[1:,:],
                                   zero_line=True)

### 7.3.1 Comparing Models

In [0]:
print(m1.rsquared)
print(m2.rsquared)

In [0]:
print(m1.rsquared_adj)
print(m2.rsquared_adj)

In [0]:
plt.figure(figsize=(12,8))
plt.scatter(sat_df.overall, m1.fittedvalues, c='r', marker='x',
            alpha=0.5, label='m1')
plt.scatter(sat_df.overall, m2.fittedvalues, c='b', marker='x',
            alpha=0.5, label='m2')
satisfaction_range = [sat_df.overall.min(), sat_df.overall.max()]

plt.plot(satisfaction_range, satisfaction_range, '--k', label = 'x=y')
plt.xlabel('Observed value')
plt.ylabel('Predicted value')
plt.legend()

In [0]:
from statsmodels.stats import anova as sms_anova
sms_anova.anova_lm(m1,m2)

### 7.3.2 Using a Model to Make Predictions

In [0]:
m2.params.Intercept + m2.params.rides*100 + m2.params.games*100\
  + m2.params.wait*100 + m2.params.clean*100

In [0]:
m2.params.Intercept + (m2.params[1:].sum() * 100)

In [0]:
m2.predict(sat_df.head(10))

In [0]:
m2.fittedvalues[:10]

In [0]:
m2.predict({'rides': 100,
            'games': 100,
            'wait': 100,
            'clean': 100})

### 7.3.3 Standardizing the Predictors

In [0]:
((sat_df.rides - sat_df.rides.mean())/sat_df.rides.std()).head(10)

In [0]:
sat_df.head()

In [0]:
sat_df_scaled = sat_df.copy()
idx = ['clean', 'games', 'rides', 'wait', 'log_dist', 'overall']
sat_df_scaled[idx] = (sat_df[idx] - sat_df[idx].mean(axis=0))\
  /sat_df[idx].std(axis=0)

In [0]:
sat_df_scaled.head()

In [0]:
sat_df_scaled[idx].describe().round(2)

## 7.4 Using Factors as Predictors

In [0]:
m3 = smf.ols('overall ~ rides + games + wait + clean + weekend'
             ' + log_dist + num_child', data=sat_df_scaled).fit()
m3.summary()

In [0]:
dummy_vals = pd.get_dummies(sat_df_scaled.num_child, prefix='num_child')
dummy_vals.head()

In [0]:
sat_df_child_factor = sat_df_scaled.join(dummy_vals)

In [0]:
m4 = smf.ols('overall ~ rides + games + wait + clean + log_dist'
             '+ num_child_0 + num_child_1 + num_child_2 + num_child_3'
             '+ num_child_4 + num_child_5',
             data=sat_df_child_factor).fit()
m4.summary()

In [0]:
sat_df_scaled['has_child'] = sat_df_scaled.num_child.apply(lambda x:
                                                           x > 0)
m5 = smf.ols('overall ~ rides + games + wait + clean + log_dist'
             '+ has_child', data=sat_df_scaled).fit()
m5.summary()

## 7.5 Interaction Terms

In [0]:
m6 = smf.ols('overall ~ rides + games + wait + clean + log_dist'
             '+ has_child + rides:has_child + games:has_child'
             '+ wait:has_child + clean:has_child + rides:weekend'
             '+ games:weekend + wait:weekend + clean:weekend',
             data=sat_df_scaled).fit()
m6.summary()

In [0]:
m7 = smf.ols('overall ~ rides + games + wait + clean + log_dist'
             '+ has_child + wait:has_child',
             data=sat_df_scaled).fit()
m7.summary()

In [0]:
chapter6.plot_confidence_intervals(m7.params[1:], m7.conf_int().iloc[1:],
                       zero_line=True)